# Advanced Agents: Model Context Protocol (MCP) Lab
Welcome to the Model Context Protocol (MCP) lab! In this hands-on session, you'll learn about MCP - an open protocol released by Anthropic that standardizes how applications provide context to LLMs.

## What is MCP?
Think of MCP like a USB-C port for AI applications. Just as USB-C provides a standardized way to connect your devices to various peripherals and accessories, MCP provides a standardized way to connect AI models to different data sources and tools.

The concept of MCP is simple. Create an open standard for how LLMs communicate with tools, resources, and prompts. As you may have noticed, nothing is standardized across generative AI. Different frameworks have different tool definitions, each API provider has a different API definition (even message definition), frameworks all do things differently. It ends up creating a lot of effort to make things all play together nicely. 

MCP aims to solve at least part of that puzzle. By creating an open specification for how people should define tools, prompts, and resources, we can start making pluggable and reusable agent components.

## What We'll Build
In this lab, we will:
1. Create an MCP Server hosting our custom tools
2. Set up an MCP Client which will consume our tool server
3. Integrate with other open source MCP servers

By the end of this lab, you'll understand how MCP creates a unified interface for LLMs to access external data and functionality, making your AI applications more modular, maintainable, and interoperable.
Let's get started!

# Build an MCP Server
MCP servers are simple using the python SDK. Define your tools and add type annotations. Then build and run the server. once started, FastMCP (part of the SDK) will automatically convert your tools into an MCP compatible server that can used by an agent. Servers can either be local (connected over STDIO) or remote (connected over HTTP SSE). They can also be written in different languages. For example, lots of servers are written in Typescript. The ones we'll write here are Python for consistency, but we'll also import 3P servers that are written in a variety of languages. 

The two MCP standard APIs we'll be playing with today are list-tools and call-tool. 

Here's a simple MCP server with two tools to call a weather API to get alerts and weather.

In [ ]:
from typing import Dict, Any
import requests
from mcp.server.fastmcp import FastMCP
import logging

class WeatherToolServer:
    """A simple MCP server providing weather-related tools."""
    
    def __init__(self, name="weather"):
        self.mcp = FastMCP(name)
        self.API_BASE = "https://api.weather.gov"
        self._register_tools()
    
    def _make_request(self, url: str) -> Dict[str, Any]:
        """Make a request to the weather API."""
        headers = {"User-Agent": "weather-app/1.0", "Accept": "application/geo+json"}
        try:
            logging.debug(f"Making request to {url}")
            response = requests.get(url, headers=headers, timeout=10.0)
            logging.debug(f"Response status: {response.status_code}")
            response.raise_for_status()
            return response.json()
        except Exception as e:
            logging.error(f"Error in API request: {str(e)}")
            return {}
    
    def _register_tools(self):
        """Register all weather tools with the MCP server."""
        
        @self.mcp.tool()
        def get_alerts(state: str) -> str:
            """Get weather alerts for a US state.
            
            Args:
                state: Two-letter US state code (e.g. CA, NY)
            """
            data = self._make_request(f"{self.API_BASE}/alerts/active/area/{state}")
            
            if not data.get("features"):
                return "No active alerts for this state."
                
            alerts = []
            for feature in data["features"]:
                props = feature["properties"]
                alerts.append(f"Event: {props.get('event')}\nArea: {props.get('areaDesc')}\nSeverity: {props.get('severity')}")
            
            return "\n---\n".join(alerts)
        
        @self.mcp.tool()
        def get_forecast(latitude: float, longitude: float) -> str:
            """Get weather forecast for a location.
            
            Args:
                latitude: Latitude of the location
                longitude: Longitude of the location
            """
            # Get points data and extract forecast URL
            points_data = self._make_request(f"{self.API_BASE}/points/{latitude},{longitude}")
            if not points_data:
                return "Unable to fetch forecast data for this location."
                
            forecast_url = points_data.get("properties", {}).get("forecast", "")
            if not forecast_url:
                return "Forecast URL not available."
                
            # Get the actual forecast
            forecast_data = self._make_request(forecast_url)
            if not forecast_data:
                return "Unable to fetch forecast."
                
            # Format just the essential information
            result = []
            for period in forecast_data.get("properties", {}).get("periods", [])[:3]:
                result.append(f"{period['name']}: {period['temperature']}°{period['temperatureUnit']}, {period['shortForecast']}")
                
            return "\n".join(result)
    
    def get_server(self):
        """Return the configured MCP server."""
        return self.mcp


# Leave this commented out, but this is how you would run the server
# app = WeatherToolServer().get_server()
# if __name__ == "__main__":
#     app.run(transport="stdio")

# Running the server
You can run the server in a couple ways. In this notebook, we'll run it with nest_asyncio. You can also debug it using the mcp run dev command. This will open up an inspector on your local host and you can tinker around with the server. 

For unit/integ tests, we recommend you just call the server object directly and validate the code. 

```bash
$ uv run mcp dev weather_server.py

# Expected output of the command.
Spawned stdio transport
Connected MCP client to backing server transport
Created web app transport
Created web app transport
Set up MCP proxy
🔍 MCP Inspector is up and running at http://127.0.0.1:6274 🚀
```

This will spin up the MCP Inspector locally on port 6274. Inspector is teh easiest way to debug your server locally at the moment. 

## How we'll test. 
In this next section we'll just call the server APIs programmatically using the FastMCP object created from WeatherToolServer().get_server().

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

Lets dump out the tool defintions from our MCP server. 

In [ ]:
# It's best practice to import as MCPTool to avoid confusion with other tool definitions from different places
# Including other frameworks or even our own. 
from mcp import Tool as MCPTool

from typing import List

mcp_server: FastMCP = WeatherToolServer().get_server()

# Define an async function to get tools
async def get_tools():
    tools: List[MCPTool] = await mcp_server.list_tools()
    for tool in tools:
        print(tool.model_dump_json(indent=2))

# Run the async function
asyncio.run(get_tools())

Nice, it looks like our tool server is returning our function definitions correctly! Now lets try to invoke the tool and test out the server using the mcp object. 

In [ ]:
# It's best practice to import as MCPTool to avoid confusion with other tool definitions from different places
# Including other frameworks or even our own. 
from mcp.types import TextContent as MCPTextContent, ImageContent as MCPImageContent, EmbeddedResource as MCPEmbeddedResource
from typing import List, Any

mcp_server: FastMCP = WeatherToolServer().get_server()

# Define an async function to get tools
async def call_tool(name: str, arguments: Dict[str, Any]) -> None:
    results: List[MCPTextContent | MCPImageContent | MCPEmbeddedResource] = await mcp_server.call_tool(name, arguments)
    for result in results:
        print(result.model_dump_json(indent=2))

# Run the async function
asyncio.run(call_tool("get_alerts", {"state": "CA"}))

# Create an MCP Client
Nice! We've now tested our server code and can see the MCP protocol in action. There's nothing too complicated about it. It's just a common interface (abstraction) that the protocol dictates. 

## Connecting a Client to an MCP Server
MCP Servers can connect in two ways, either over STDIO and HTTP SSE. With STDIO running your own MCP server, you specify a server config pointing to the absolute file path for the server and a command to run. MCP locally will spin up that server as a subprocess and maintain a connection with it. This is the path we'll take. Like shown above, the MCP spec is changing to make remote servers easier to use. As of 4/5/2025, we recommend running the servers locally until the spec updates propagate into the SDKs. Without that change, you end up doing unnatural things and writing shims to get it to function in a production system. 

First lets get the absolute path to our weather server and setup our server params using MCPs StdioServerParameters class

In [ ]:
from pydantic_ai import Agent
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# We need the full path to the weather server
import os
# Get the absolute path to the weather server
current_dir = os.getcwd()
weather_server_path = os.path.join(current_dir, 'mcp_servers', 'weather_server', 'weather_server.py')

server_params = StdioServerParameters(  
    command = 'uv',
    args=['run', 'mcp', 'run', weather_server_path]
)

Next lets test out the connection using a test function. We'll use the mcp SDK to directly connect to the server without any agent framework.

stdio_client and ClientSession are async so we need to wrap the call in asyncio so that it runs in a jupyter notebook. 

In [ ]:
from mcp import ClientSession
from mcp.client.stdio import stdio_client
from mcp.types import ListToolsResult

async def test_mcp_client():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            print('initializing session')
            await session.initialize()
            tools: ListToolsResult = await session.list_tools()

            # Print out the tools from our client. 
            for t in tools.tools:
                print(t.name)
                print(t.description)
                print(t.inputSchema)
                print('-'*100)


asyncio.run(test_mcp_client())

# Connect our MCP Servers to an Agent
We could re-use the agent we created from scratch in module 3, however we have lots of useful frameworks that now work interchangably with the rest of our code. I don't see the point in reinventing the wheel if the framework does what we need out of the box. Lets use the PydanticAI agent we created in the previous workshop and leverage it's MCP features. 

PydanticAI comes with two ways to connect to MCP servers:
* MCPServerHTTP which connects to an MCP server using the HTTP SSE transport
* MCPServerStdio which runs the server as a subprocess and connects to it using the stdio transport

In [ ]:
# First we wrap our own MCP server in a MCPServerStdio object
from pydantic_ai.mcp import MCPServerStdio as PyAIMCPServerStdio

custom_weather_mcp_server = PyAIMCPServerStdio(  
    command = 'uv',
    args=['run', 'mcp', 'run', weather_server_path]
)


In [ ]:
from pydantic_ai import Agent as PyAIAgent

# Add the MCP Server params to the agent.
weather_agent: PyAIAgent = PyAIAgent(
    'bedrock:us.anthropic.claude-3-5-haiku-20241022-v1:0',
    system_prompt='You are a helpful assistant.',
    mcp_servers=[custom_weather_mcp_server]
)

# We can reuse this to run multiple agents with different servers.
async def run_pydantic_ai_mcp_client(user_msg: str,agent: PyAIAgent):
    async with agent.run_mcp_servers():
        result = await agent.run(user_msg)
    print(result.data)

# Run the agent with a user message.
user_msg = 'Can you show me any weather alerts for California?'
asyncio.run(run_pydantic_ai_mcp_client(user_msg, weather_agent))

# Conclusion
In this lab we successfully created an MCP server, connected it to an MCP client (using PydanticAI), and were able to augment our LLM with this custom server.
